<a href="https://colab.research.google.com/github/edwin2017/BotOrderShop/blob/master/%D0%A0%D0%B0%D0%B7%D0%B1%D0%BE%D1%80_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D1%8F_Python_Bootcamp_Day_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задание 1
Улучшите функцию приветствия, добавив имя пользователя.

**Подсказка 1**

В каком атрибуте хранится имя пользователя, можно посмотреть в документации к API Телеграма: https://core.telegram.org/bots/api#message

**Подсказка 2**

* Имя пользователя хранится в атрибуте `message.chat.first_name` https://core.telegram.org/bots/api#chat
* Текст из разных кусочков мы собираем с помощью оператора +: `"Hello, " + message.chat.first_name`

In [ ]:
# Загружаем библиотеки
!pip install pyTelegramBotAPI

In [ ]:
# Подключаем библиотеку
import telebot

# Здесь нужно вставить токен, который дал BotFather при регистрации
# Пример: token = '2007628239:AAEF4ZVqLiRKG7j49EC4vaRwXjJ6DN6xng8'
token = '<ваш токен>'  # <<< Ваш токен

# В этой строчке мы заводим бота и даем ему запомнить токен
bot = telebot.TeleBot(token)

# Пишем первую функцию, которая отвечает "Привет" на команду /start
# Все функции общения приложения с ТГ спрятаны в функции под @
@bot.message_handler(commands=['start'])
def say_hi(message):
    bot.send_message(message.chat.id, 'Привет, ' + message.chat.first_name)

# Запускаем бота. Он будет работать до тех пор, пока работает ячейка (крутится значок слева).
# Остановим ячейку - остановится бот
bot.polling()

## Задание 2

**Подсказка**

* Стикеры отправляются в формате **.webp**
* Стикер отправляется с помощью функции `send_sticker`
* Пример использования можно посмотреть
в описании библиотеки https://github.com/eternnoir/pyTelegramBotAPI

In [ ]:
!pip install pyTelegramBotAPI SpeechRecognition pydub

In [ ]:
# Загрузка изображения для демо
import urllib

url = "https://drive.google.com/uc?export=view&id=1WDN5RXcYQHiUT4JVujQ2VSwr7p0XLlYX"
filename = "skillbox_sticker.webp"
urllib.request.urlretrieve(url, filename)

('skillbox_sticker.webp', <http.client.HTTPMessage at 0x7fcfc42f6be0>)

In [ ]:
import os
import telebot
import speech_recognition
from pydub import AudioSegment


# Ниже нужно вставить токен, который дал BotFather при регистрации
# Пример: token = '2007628239:AAEF4ZVqLiRKG7j49EC4vaRwXjJ6DN6xng8'
token = '<ваш токен>'  # <<< Ваш токен

bot = telebot.TeleBot(token)


def oga2wav(filename):
    # Конвертация формата файлов
    new_filename = filename.replace('.oga', '.wav')
    audio = AudioSegment.from_file(filename)
    audio.export(new_filename, format='wav')
    return new_filename


def recognize_speech(oga_filename):
    # Перевод голоса в текст + удаление использованных файлов
    wav_filename = oga2wav(oga_filename)
    recognizer = speech_recognition.Recognizer()

    with speech_recognition.WavFile(wav_filename) as source:     
        wav_audio = recognizer.record(source)

    text = recognizer.recognize_google(wav_audio, language='ru')

    if os.path.exists(oga_filename):
        os.remove(oga_filename)

    if os.path.exists(wav_filename):
        os.remove(wav_filename)

    return text


def download_file(bot, file_id):
    # Скачивание файла, который прислал пользователь
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    filename = file_id + file_info.file_path
    filename = filename.replace('/', '_')
    with open(filename, 'wb') as f:
        f.write(downloaded_file)
    return filename


@bot.message_handler(commands=['start'])
def say_hi(message):
    # Функция, отправляющая "Привет" в ответ на команду /start
    bot.send_message(message.chat.id, 'Привет')
    sticker = open('skillbox_sticker.webp', 'rb')
    bot.send_sticker(message.chat.id, sticker)
    sticker.close()


@bot.message_handler(content_types=['voice'])
def transcript(message):
    # Функция, отправляющая текст в ответ на голосовое
    filename = download_file(bot, message.voice.file_id)
    text = recognize_speech(filename)
    bot.send_message(message.chat.id, text)


# Запускаем бота. Он будет работать до тех пор, пока работает ячейка (крутится значок слева).
# Остановим ячейку - остановится бот
bot.polling()